# 🧠 Maze Navigation: NEAT vs DQN Training Comparison

This notebook provides a complete workflow for training, evaluating, and comparing both NEAT and DQN approaches for maze navigation.

## Table of Contents
1. Setup and Imports  
2. Environment Exploration  
3. Train NEAT Agent  
4. Train DQN Agent  
5. Compare Performance  
6. Visualize Decision Making  
7. Robustness Testing  
8. Conclusions

In [ ]:
# ============================================================================
# CELL 1: Setup and Imports
# ============================================================================

import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Custom imports
from env.maze_env import MazeEnv
from neuroevolution.neat_solver import NEATMazeSolver, create_neat_config
from reinforcement_learning.dqn_solver import DQNMazeSolver
from analysis.visualize_training import TrainingVisualizer
from analysis.robustness_tests import RobustnessTestSuite

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("✅ All imports successful!")
print(f"NumPy version: {np.__version__}")

## 2. Environment Exploration

Let's explore the maze environment and understand its characteristics.

In [ ]:
# ============================================================================
# CELL 2: Environment Exploration
# ============================================================================

env = MazeEnv()

print("Maze Configuration:")
print(f"  Size: {env.height}x{env.width}")
print(f"  Start: {env.start_pos}")
print(f"  Goal: {env.goal_pos}")
print(f"  Observation Space: {env.observation_space}")
print(f"  Action Space: {env.action_space}")

fig, ax = plt.subplots(figsize=(8, 8))
env.reset()
env._draw_maze(ax)
ax.set_title("Initial Maze Configuration", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTesting random agent for 100 steps...")
obs, _ = env.reset()
total_reward = 0
for step in range(100):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    if terminated or truncated:
        break

print(f"Random agent performance:")
print(f"  Steps taken: {step + 1}")
print(f"  Total reward: {total_reward:.2f}")
print(f"  Final distance: {info['distance_to_goal']:.2f}")
print(f"  Cells explored: {info['unique_cells_visited']}")

## 3. Train NEAT Agent

Training the Neuroevolution approach using the NEAT algorithm.

In [ ]:
# ============================================================================
# CELL 3: Train NEAT Agent
# ============================================================================

print("="*60)
print("TRAINING NEAT AGENT")
print("="*60)

config_path = create_neat_config('../neuroevolution/config-neat.txt')
neat_solver = NEATMazeSolver(config_path, log_dir='../logs/neat')

print("\nStarting NEAT training...")
print("This will take approximately 20-30 minutes on CPU\n")

winner = neat_solver.train(generations=50)

print("\n✅ NEAT training complete!")
print(f"Best fitness: {neat_solver.best_fitness:.2f}")

neat_solver.visualize_training()

print("\nEvaluating best NEAT genome...")
neat_results = neat_solver.evaluate_best(render=False, num_episodes=10)

success_rate = sum(r['reached_goal'] for r in neat_results) / len(neat_results)
avg_steps = np.mean([r['steps'] for r in neat_results])
avg_reward = np.mean([r['reward'] for r in neat_results])

print(f"\nNEAT Evaluation Results:")
print(f"  Success Rate: {success_rate:.1%}")
print(f"  Avg Steps: {avg_steps:.1f}")
print(f"  Avg Reward: {avg_reward:.2f}")

## 4. Train DQN Agent

Training the Deep Q-Network (DQN) agent.

In [ ]:
# ============================================================================
# CELL 4: Train DQN Agent
# ============================================================================

print("="*60)
print("TRAINING DQN AGENT")
print("="*60)

env_dqn = MazeEnv()
dqn_solver = DQNMazeSolver(env_dqn, log_dir='../logs/dqn')

print(f"\nUsing device: {dqn_solver.device}")
print("This will take approximately 30-45 minutes on CPU\n")

dqn_solver.train(num_episodes=500, verbose=True)

print("\n✅ DQN training complete!")
print(f"Best reward: {dqn_solver.best_reward:.2f}")

dqn_solver.visualize_training()

print("\nEvaluating trained DQN agent...")
dqn_results = dqn_solver.evaluate(num_episodes=10, render=False)

dqn_success_rate = sum(r['reached_goal'] for r in dqn_results) / len(dqn_results)
dqn_avg_steps = np.mean([r['steps'] for r in dqn_results])
dqn_avg_reward = np.mean([r['reward'] for r in dqn_results])

print(f"\nDQN Evaluation Results:")
print(f"  Success Rate: {dqn_success_rate:.1%}")
print(f"  Avg Steps: {dqn_avg_steps:.1f}")
print(f"  Avg Reward: {dqn_avg_reward:.2f}")

## 5. Compare Performance

In [ ]:
# ============================================================================
# CELL 5: Compare Performance
# ============================================================================

print("="*60)
print("PERFORMANCE COMPARISON")
print("="*60)

import pandas as pd

comparison_data = {
    'Metric': ['Success Rate', 'Avg Steps', 'Avg Reward', 'Training Time'],
    'NEAT': [f"{success_rate:.1%}", f"{avg_steps:.1f}", f"{avg_reward:.2f}", "~30 min"],
    'DQN': [f"{dqn_success_rate:.1%}", f"{dqn_avg_steps:.1f}", f"{dqn_avg_reward:.2f}", "~45 min"]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n", df_comparison.to_string(index=False))

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Success Rate
ax = axes[0]
methods = ['NEAT', 'DQN']
success_rates = [success_rate, dqn_success_rate]
bars = ax.bar(methods, success_rates, color=['#3498db', '#e74c3c'], alpha=0.7)
ax.set_ylabel('Success Rate')
ax.set_title('Success Rate Comparison', fontweight='bold')
ax.set_ylim(0, 1.1)
for bar, rate in zip(bars, success_rates):
    ax.text(bar.get_x() + bar.get_width()/2., rate, f'{rate:.1%}', ha='center', va='bottom', fontweight='bold')

# Average Steps
ax = axes[1]
steps = [avg_steps, dqn_avg_steps]
bars = ax.bar(methods, steps, color=['#3498db', '#e74c3c'], alpha=0.7)
ax.set_ylabel('Average Steps')
ax.set_title('Efficiency Comparison', fontweight='bold')
for bar, step in zip(bars, steps):
    ax.text(bar.get_x() + bar.get_width()/2., step, f'{step:.0f}', ha='center', va='bottom', fontweight='bold')

# Average Reward
ax = axes[2]
rewards = [avg_reward, dqn_avg_reward]
bars = ax.bar(methods, rewards, color=['#3498db', '#e74c3c'], alpha=0.7)
ax.set_ylabel('Average Reward')
ax.set_title('Reward Comparison', fontweight='bold')
for bar, reward in zip(bars, rewards):
    ax.text(bar.get_x() + bar.get_width()/2., reward, f'{reward:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Visualize Decision Making

In [ ]:
# ============================================================================
# CELL 6: Visualize Decision Making
# ============================================================================

print("="*60)
print("DECISION MAKING ANALYSIS")
print("="*60)

visualizer = TrainingVisualizer(
    neat_log_dir='../logs/neat',
    dqn_log_dir='../logs/dqn'
)

print("\nGenerating comparison dashboard...")
visualizer.create_comparison_dashboard()

print("Generating decision boundary visualization...")
visualizer.visualize_decision_boundaries()

print("Creating live comparison visualization...")
visualizer.create_live_comparison()

print("\n✅ All visualizations generated!")

## 7. Robustness Testing

In [ ]:
# ============================================================================
# CELL 7: Robustness Testing
# ============================================================================

print("="*60)
print("ROBUSTNESS TESTING")
print("="*60)

test_suite = RobustnessTestSuite(
    neat_model_path='../logs/neat/best_genome_gen_50.pkl',
    dqn_model_path='../logs/dqn/best_model.pth',
    neat_config_path='../neuroevolution/config-neat.txt'
)

print("\n[Test 1/3] Noise Sensitivity Analysis...")
noise_results = test_suite.test_noise_sensitivity(
    noise_levels=[0.0, 0.05, 0.1, 0.15, 0.2, 0.3]
)

print("\n[Test 2/3] Generalization Testing...")
generalization_results = test_suite.test_generalization(num_test_mazes=10)

print("\n[Test 3/3] Failure Mode Analysis...")
failure_results = test_suite.test_failure_modes()

print("\n[Computing Overall Scores...]")
robustness_scores = test_suite.compute_robustness_score()

print("\n" + "="*60)
print("ROBUSTNESS SCORES SUMMARY")
print("="*60)

for agent in ['neat', 'dqn']:
    print(f"\n{agent.upper()}:")
    for metric, score in robustness_scores[agent].items():
        print(f"  {metric.replace('_', ' ').title()}: {score:.2f}/100")

## 8. Conclusions

In [ ]:
# ============================================================================
# CELL 8: Conclusions
# ============================================================================

print("="*60)
print("NOTEBOOK COMPLETE!")
print("="*60)
print("\n📊 All results saved to respective log directories")
print("📈 Visualizations saved to analysis/ directory")
print("🎯 Models saved and ready for deployment")
print("\nThank you for exploring NEAT vs DQN maze navigation!")
print("For questions or contributions, visit:")
print("https://github.com/yourusername/maze-nav-project")